In [ ]:

from google.colab import files
uploaded = files.upload()


In [ ]:
!pip install -r requirements.txt


In [ ]:
import os
os.environ["HF_API_KEY"] = ""


In [ ]:
from huggingface_hub import login
login(os.environ["HF_API_KEY"])


In [ ]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

import os
import io
import IPython.display
from PIL import Image
import base64

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv() # read local .env file

from huggingface_hub import login

login(os.environ['HF_API_KEY'])

In [ ]:
import pandas as pd

suppliers_data = {
    "name": [
        "Montreal Ice Cream Co",
        "Brain Freeze Brothers",
        "Toronto Gelato Ltd",
        "Buffalo Scoops",
        "Vermont Creamery",
    ],
    "location": [
        "Montreal, QC",
        "Burlington, VT",
        "Toronto, ON",
        "Buffalo, NY",
        "Portland, ME",
    ],
    "distance_km": [120, 85, 400, 220, 280],
    "canadian": [True, False, True, False, False],
    "price_per_liter": [1.95, 1.91, 1.82, 2.43, 2.33],
    "tasting_fee": [0, 12.50, 30.14, 42.00, 0.20],
}

data_description = """Suppliers have an additional tasting fee: that is a fixed fee applied to each order to taste the ice cream."""
suppliers_df = pd.DataFrame(suppliers_data)
suppliers_df

,name,location,distance_km,canadian,price_per_liter,tasting_fee
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20


## Setup tools

In [ ]:
import numpy as np

def calculate_daily_supplier_price(row):
    order_volume = 30

    # Calculate raw product cost
    product_cost = row["price_per_liter"] * order_volume

    # Calculate transport cost
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    transport_cost = row["distance_km"] * cost_per_km * trucks_needed

    # Calculate tariffs for ice cream imported from Canada
    tariff = product_cost * np.pi / 50 * row["canadian"]

    # Get total cost
    total_cost = product_cost + transport_cost + tariff + row["tasting_fee"]
    return total_cost

suppliers_df["daily_price"] = suppliers_df.apply(calculate_daily_supplier_price, axis=1)
display(suppliers_df)

# Let's remove this column now.
suppliers_df = suppliers_df.drop("daily_price", axis=1)

,name,location,distance_km,canadian,price_per_liter,tasting_fee,daily_price
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00,206.175663
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50,171.800000
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14,568.170619
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00,378.900000
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20,406.100000


In [ ]:
# First we make a few tools
from smolagents import tool

@tool
def calculate_transport_cost(distance_km: float, order_volume: float) -> float:
    """
    Calculate transportation cost based on distance and order size.
    Refrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.

    Args:
        distance_km: the distance in kilometers
        order_volume: the order volume in liters
    """
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    return distance_km * cost_per_km * trucks_needed


@tool
def calculate_tariff(base_cost: float, is_canadian: bool) -> float:
    """
    Calculates tariff for Canadian imports. Returns the tariff only, not the total cost.
    Assumes tariff on dairy products from Canada is worth 2 * pi / 100, approx 6.2%

    Args:
        base_cost: the base cost of goods, not including transportation cost.
        is_canadian: wether the import is from Canada.
    """
    if is_canadian:
        return base_cost * np.pi / 50
    return 0

In [ ]:
def calculate_transport_cost(distance_km: float, order_volume: float) -> float:
    """
    Calculate transportation cost based on distance and order size.
    Refrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.

    Args:
        distance_km: the distance in kilometers
        order_volume: the order volume in liters
    """
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    return distance_km * cost_per_km * trucks_needed

print(calculate_transport_cost(10,50))

12.0


In [ ]:
calculate_transport_cost.description

'Calculate transportation cost based on distance and order size.\nRefrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.'

In [ ]:
calculate_tariff.description

'Calculates tariff for Canadian imports. Returns the tariff only, not the total cost.\nAssumes tariff on dairy products from Canada is worth 2 * pi / 100, approx 6.2%'

## Setup the Model

In [ ]:
from smolagents import HfApiModel, CodeAgent
from helper import get_huggingface_token

model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    provider="together", # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.1
)

In [ ]:
url = "http://<IP>:11434/api/generate"

payload = {
    "model": "granite:latest",  # Or the model name you have
    "prompt": "Hello, how are you?",
    "temperature": 0.1,
    "num_predict": 4096
}

try:
    response = requests.post(url, json=payload)
    response.raise_for_status()
    print("API Response:", response.json())
except requests.exceptions.RequestException as e:
    print(f"Error calling the API: {e}")


In [ ]:
OLLAMA_HOST=0.0.0.0 ollama serve


In [ ]:
import requests

class OllamaModel:
    def __init__(self, model="granite:code", base_url="http://<IP>:11434", temperature=0.1, max_tokens=4096):
        self.model = model
        self.url = f"{base_url}/api/generate"
        self.temperature = temperature
        self.max_tokens = max_tokens

    def run(self, prompt: str) -> str:
        payload = {
            "model": self.model,
            "prompt": prompt,
            "temperature": self.temperature,
            "stream": True,
            "num_predict": self.max_tokens
        }
        response = requests.post(self.url, json=payload)
        response.raise_for_status()
        return response.json()["response"]

# Instantiate the model
model = OllamaModel()

# Example usage
prompt = "Write a Python function that counts the number of vowels in a string."
response = model.run(prompt)
print(response)



In [ ]:
# from smolagents import HfApiModel, CodeAgent  # ⛔️ Not needed anymore
from ollama_model import OllamaModel
from helper import get_huggingface_token  # Keep if you're still using HF for anything else

model = OllamaModel(
    model="granite:latest",  # or "mistral", "llama2", etc.
    base_url="http://<IP>:11434",  # replace with your actual VM IP
    temperature=0.1,
    max_tokens=4096
)


## Setup the Code Agent


In [ ]:
agent = CodeAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=10,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2
)
agent.logger.console.width=66

In [ ]:
agent.run(
    """Can you get me the transportation cost for 50 liters
    of ice cream over 10 kilometers?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the transportation cost for 50 liters           │
│     of ice cream over 10 kilometers?                           │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will use the `calculate_transport_cost` tool to        
calculate the cost.                                               
Code:                                                             
```py                                                             
transport_cost = calculate_transport_cost(distance_km=10,         
order_volume=50)                                                  
print(transport_cost)                                             
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  transport_cost = calculate_transport_cost(distance_km=10,       
  order_volume=50)                                                
  print(transport_cost)                                           
 ────────────────────────────────────────────────────────────────

Execution logs:
12.0

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: The transportation cost for 50 liters of ice cream over  
10 kilometers is 12.0. I will provide this as the final answer.   
Code:                                                             
```py                                                             
final_answer(12.0)                                                
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(12.0)                                              
 ────────────────────────────────────────────────────────────────

Out - Final answer: 12.0

[Step 2: Duration 1.24 seconds| Input tokens: 4,624 | Output 
tokens: 102]

12.0

In [ ]:
task = """Here is a dataframe of different ice cream suppliers.
Could you give me a comparative table (as a dataframe) of the total
daily price for getting daily ice cream delivery from each of them,
given that we need exactly 30 liters of ice cream per day? Take
into account transportation cost and tariffs.
"""
agent.logger.level = 3 # Lower verbosity level
agent.run(
    task,
    additional_args={"suppliers_data": suppliers_df, "data_description": data_description},
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Here is a dataframe of different ice cream suppliers.          │
│ Could you give me a comparative table (as a dataframe) of the  │
│ total                                                          │
│ daily price for getting daily ice cream delivery from each of  │
│ them,                                                          │
│ given that we need exactly 30 liters of ice cream per day?     │
│ Take                                                           │
│ into account transportation cost and tariffs.                  │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I need to calculate the total daily price for each       
supplier, taking into account the cost of the ice cream, the      
transportation cost, and the tariff if applicable. I will use the 
`calculate_transport_cost` and `calculate_tariff` functions to do 
this. I will create a new column in the dataframe for the total   
daily price.                                                      
Code:                                                             
```py                                                             
import pandas as pd                                               
                                                                  
# Convert the provided data into a DataFrame                      
suppliers_df = pd.DataFrame(suppliers_data)                       
                                                                  
# Define the order volume                                         
order_volume = 30                                                 
                                                                  
# Calculate the total daily price for each supplier               
def calculate_total_daily_price(row):                             
    # Calculate the cost of the ice cream                         
    ice_cream_cost = row['price_per_liter'] * order_volume        
    # Calculate the transportation cost                           
    transport_cost =                                              
calculate_transport_cost(distance_km=row['distance_km'],          
order_volume=order_volume)                                        
    # Calculate the tariff if the supplier is Canadian            
    tariff = calculate_tariff(base_cost=ice_cream_cost,           
is_canadian=row['canadian'])                                      
    # Calculate the total daily price                             
    total_daily_price = ice_cream_cost + transport_cost + tariff +
row['tasting_fee']                                                
    return total_daily_price                                      
                                                                  
# Apply the function to each row in the DataFrame                 
suppliers_df['total_daily_price'] =                               
suppliers_df.apply(calculate_total_daily_price, axis=1)           
                                                                  
# Print the resulting DataFrame                                   
print(suppliers_df[['name', 'total_daily_price']])                
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  import pandas as pd                                             
                                                                  
  # Convert the provided data into a DataFrame                    
  suppliers_df = pd.DataFrame(suppliers_data)                     
                                                                  
  # Define the order volume                                       
  order_volume = 30                                               
                                                                  
  # Calculate the total daily price for each supplier             
  def calculate_total_daily_price(row):                           
      # Calculate the cost of the ice cream                       
      ice_cream_cost = row['price_per_liter'] * order_volume      
      # Calculate the transportation cost                         
      transport_cost =                                            
  calculate_transport_cost(distance_km=row['distance_km'],        
  order_volume=order_volume)                                      
      # Calculate the tariff if the supplier is Canadian          
      tariff = calculate_tariff(base_cost=ice_cream_cost,         
  is_canadian=row['canadian'])                                    
      # Calculate the total daily price                           
      total_daily_price = ice_cream_cost + transport_cost +       
  tariff + row['tasting_fee']                                     
      return total_daily_price                                    
                                                                  
  # Apply the function to each row in the DataFrame               
  suppliers_df['total_daily_price'] =                             
  suppliers_df.apply(calculate_total_daily_price, axis=1)         
                                                                  
  # Print the resulting DataFrame                                 
  print(suppliers_df[['name', 'total_daily_price']])              
 ────────────────────────────────────────────────────────────────

Execution logs:
                    name  total_daily_price
0  Montreal Ice Cream Co         206.175663
1  Brain Freeze Brothers         171.800000
2     Toronto Gelato Ltd         568.170619
3         Buffalo Scoops         378.900000
4       Vermont Creamery         406.100000

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: I have successfully calculated the total daily price for 
each supplier and added it as a new column to the dataframe. Now, 
I can return the final answer.                                    
Code:                                                             
```py                                                             
final_answer(suppliers_df[['name', 'total_daily_price']])         
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(suppliers_df[['name', 'total_daily_price']])       
 ────────────────────────────────────────────────────────────────

Out - Final answer:                     name  total_daily_price
0  Montreal Ice Cream Co         206.175663
1  Brain Freeze Brothers         171.800000
2     Toronto Gelato Ltd         568.170619
3         Buffalo Scoops         378.900000
4       Vermont Creamery         406.100000

[Step 2: Duration 1.84 seconds| Input tokens: 5,749 | Output 
tokens: 365]

,name,total_daily_price
0,Montreal Ice Cream Co,206.175663
1,Brain Freeze Brothers,171.800000
2,Toronto Gelato Ltd,568.170619
3,Buffalo Scoops,378.900000
4,Vermont Creamery,406.100000


In [ ]:
task = """From the dataframe above, which supplier is the cheapest? Consider, I need to travel 100 kilometers to get ice cream,
and 100 ltr of ice cream.
"""
agent.logger.level = 3 # Lower verbosity level
agent.run(
    task,
    additional_args={"suppliers_data": suppliers_df, "data_description": data_description},
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ From the dataframe above, which supplier is the cheapest?      │
│ Consider, I need to travel 100 kilometers to get ice cream,    │
│ and 100 ltr of ice cream.                                      │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will calculate the total cost for each supplier,       
including the transportation cost and the tariff if applicable.   
Then, I will find the supplier with the lowest total cost.        
Code:                                                             
```py                                                             
import pandas as pd                                               
                                                                  
# Load the data                                                   
data = suppliers_data                                             
                                                                  
# Calculate the total cost for each supplier                      
def calculate_total_cost(row):                                    
    distance_km = row['distance_km']                              
    order_volume = 100  # 100 ltr of ice cream                    
    price_per_liter = row['price_per_liter']                      
    tasting_fee = row['tasting_fee']                              
    is_canadian = row['canadian']                                 
                                                                  
    # Calculate transportation cost                               
    transport_cost =                                              
calculate_transport_cost(distance_km=distance_km,                 
order_volume=order_volume)                                        
                                                                  
    # Calculate tariff                                            
    base_cost = price_per_liter * order_volume + tasting_fee      
    tariff = calculate_tariff(base_cost=base_cost,                
is_canadian=is_canadian)                                          
                                                                  
    # Calculate total cost                                        
    total_cost = base_cost + transport_cost + tariff              
    return total_cost                                             
                                                                  
# Apply the function to each row                                  
data['total_cost'] = data.apply(calculate_total_cost, axis=1)     
                                                                  
# Find the supplier with the lowest total cost                    
cheapest_supplier = data.loc[data['total_cost'].idxmin()]['name'] 
print(cheapest_supplier)                                          
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  import pandas as pd                                             
                                                                  
  # Load the data                                                 
  data = suppliers_data                                           
                                                                  
  # Calculate the total cost for each supplier                    
  def calculate_total_cost(row):                                  
      distance_km = row['distance_km']                            
      order_volume = 100  # 100 ltr of ice cream                  
      price_per_liter = row['price_per_liter']                    
      tasting_fee = row['tasting_fee']                            
      is_canadian = row['canadian']                               
                                                                  
      # Calculate transportation cost                             
      transport_cost =                                            
  calculate_transport_cost(distance_km=distance_km,               
  order_volume=order_volume)                                      
                                                                  
      # Calculate tariff                                          
      base_cost = price_per_liter * order_volume + tasting_fee    
      tariff = calculate_tariff(base_cost=base_cost,              
  is_canadian=is_canadian)                                        
                                                                  
      # Calculate total cost                                      
      total_cost = base_cost + transport_cost + tariff            
      return total_cost                                           
                                                                  
  # Apply the function to each row                                
  data['total_cost'] = data.apply(calculate_total_cost, axis=1)   
                                                                  
  # Find the supplier with the lowest total cost                  
  cheapest_supplier =                                             
  data.loc[data['total_cost'].idxmin()]['name']                   
  print(cheapest_supplier)                                        
 ────────────────────────────────────────────────────────────────

Execution logs:
Brain Freeze Brothers

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: The supplier with the lowest total cost is Brain Freeze  
Brothers. I will now provide the final answer.                    
Code:                                                             
```py                                                             
final_answer(cheapest_supplier)                                   
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(cheapest_supplier)                                 
 ────────────────────────────────────────────────────────────────

Out - Final answer: Brain Freeze Brothers

[Step 2: Duration 1.93 seconds| Input tokens: 5,602 | Output 
tokens: 324]

'Brain Freeze Brothers'

# Link parsing and finding PhD positions in AI-ML

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_text_and_links(url):
  """
  Fetches the text content of a given URL and extracts all the links within it.

  Args:
    url: The URL of the webpage to scrape.

  Returns:
    A tuple containing:
      - The text content of the webpage.
      - A list of links found within the webpage.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    soup = BeautifulSoup(response.content, "html.parser")

    # Extract text content
    text_content = soup.get_text(strip=True)

    # Extract links
    links = [link.get("href") for link in soup.find_all("a", href=True)]

    return text_content, links

  except requests.exceptions.RequestException as e:
    print(f"Error fetching URL: {e}")
    return None, None

In [ ]:
print(extract_text_and_links('https://scholaridea.com/2025/04/21/phd-and-postdoctoral-scholarships-48-at-eth-zurich-university-in-switzerland/'))

('PhD and Postdoctoral Scholarships (48) at ETH Zurich University inSkip to contentApril 25, 2025Scholar IdeaEducation, Scholarships and ResearchPrimary MenuHomeScholarshipsPhD ScholarshipsPostdoctoral ScholarshipsCountryAbu DubaiAustraliaAustriaBelgiumCanadaChinaCzech RepublicDenmarkEgyptEstoniaFinlandFranceGermanyHong KongIcelandItalyJapanKuwaitLuxembourgNetherlandsNew ZealandNorwayOmanPolandPortugalQatarRussiaSaudi ArabiaScotlandSingaporeSpainSwedenSwitzerlandTaiwanTurkeyUnited Arab EmirateUnited KingdomUnited StatesAcademic JobsHandoutsClinical ChemistryLab. TestsHematologyHematology LectureHematology NotesPractical HematologyHematology videoParasitologyUrologyHealth and FitnessComputer TipsJournal of Advanced Veterinary Research202020212022Search for:WatchHome2025April21PhD and Postdoctoral Scholarships (48) at ETH Zurich University in SwitzerlandAcademic JobsCountryPhD ScholarshipsPostdoctoral ScholarshipsScholarshipsScholarships and JobsSwitzerlandPhD and Postdoctoral Scholarshi

In [ ]:
url = "https://scholaridea.com/2025/04/21/phd-and-postdoctoral-scholarships-48-at-eth-zurich-university-in-switzerland/"


In [ ]:
from typing import Tuple

@tool
def extract_text_and_links(url: str) -> Tuple[str, list]: # a tuple containing text and links inside given url
  """
  Fetches the text content of a given URL and extracts all the links within it.

  Args:
    url: The URL of the webpage to scrape.

  Returns:
    A tuple containing:
      - The text content of the webpage.
      - A list of links found within the webpage.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    soup = BeautifulSoup(response.content, "html.parser")

    # Extract text content
    text_content = soup.get_text(strip=True)

    # Extract links
    links = [link.get("href") for link in soup.find_all("a", href=True)]

    return text_content, links

  except requests.exceptions.RequestException as e:
    print(f"Error fetching URL: {e}")
    return None, None

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[extract_text_and_links],
    max_steps=10,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2
)
agent.logger.console.width=100

In [ ]:
task = """Collect the information about the PhD positions in AI-ML or related field from the given link.
Then make a dataframe with following columns: Description, PhD Position, Location, URL
"""
agent.logger.level = 2 # Lower verbosity level
agent.run(
    task,
    additional_args={'url':url},
)